In [ ]:
def generar_serie_sintetica(longitud=500, nivel_ruido=1.0, num_cambios=3, fuerza_cambio=5.0, 
                            tipo_cambio='escalon', seed=None, graficar=True):
    """
    Genera una serie de tiempo sintética con control sobre ruido y puntos de cambio.

    Parámetros:
    -----------
    longitud : int
        Longitud de la serie de tiempo a generar (default=500)
    nivel_ruido : float
        Grado de ruido en la serie (0=sin ruido, 1=ruido moderado, >1=alto ruido) (default=1.0)
    num_cambios : int
        Número de puntos de cambio a introducir (default=3)
    fuerza_cambio : float
        Magnitud del cambio en cada punto de cambio (default=5.0)
    tipo_cambio : str
        Tipo de cambio: 'escalon' o 'pendiente' (default='escalon')
    seed : int
        Semilla para reproducibilidad (default=None)
    graficar : bool
        Si es True, muestra un gráfico de la serie (default=True)

    Retorna:
    --------
    serie : numpy.ndarray
        Serie de tiempo generada
    puntos_cambio : list
        Lista de índices donde se encuentran los puntos de cambio
    """
    if seed is not None:
        np.random.seed(seed)

    # Elegir tipo de tendencia al azar
    trend_type = np.random.choice(["creciente", "decreciente", "plana"])
    
    # Pendiente según tipo
    if trend_type == "creciente":
        slope = np.random.uniform(0.01, 0.1)
    elif trend_type == "decreciente":
        slope = np.random.uniform(-0.1, -0.01)
    else:  # plana
        slope = 0.0

    # Eje temporal
    t = np.arange(longitud)
    
    # Generar tendencia lineal
    tendencia = slope * t
  
    # Inicializar la serie con la tendencia base
    serie_base = tendencia.copy()
    serie_con_cambios = tendencia.copy()
    
    # Generar puntos de cambio aleatorios (evitando los extremos)
    puntos_cambio = []
    if num_cambios > 0:
        min_distancia = max(30, longitud // 30)  # Distancia mínima entre cambios
        espacio_disponible = list(range(min_distancia, longitud - min_distancia))
        
        for _ in range(num_cambios):
            if not espacio_disponible:
                break
            punto = np.random.choice(espacio_disponible)
            puntos_cambio.append(punto)
            espacio_disponible = [x for x in espacio_disponible 
                                if abs(x - punto) >= min_distancia]
    
    puntos_cambio.sort()
    
    # Aplicar cambios
    for i, punto in enumerate(puntos_cambio):
        direccion = 1 if np.random.random() > 0.5 else -1
        magnitud = direccion * fuerza_cambio * (0.8 + 0.4 * np.random.random())
        
        if tipo_cambio == 'escalon':
            # Cambio abrupto permanente
            serie_con_cambios[punto:] += magnitud
        
        elif tipo_cambio == 'pendiente':
            # Cambio gradual
            longitud_cambio = np.random.randint(5, min(50, longitud - punto - 10))
            fin_cambio = min(punto + longitud_cambio, longitud)
            print(fin_cambio)
            
            # Incremento progresivo
            #incremento = np.linspace(0, magnitud, fin_cambio - punto)
            #serie_con_cambios[punto:fin_cambio] += incremento
            incremento = np.linspace(0, magnitud, longitud- punto)
            serie_con_cambios[punto:longitud] += incremento
            
            # Mantener el nuevo nivel constante después del cambio
            #if fin_cambio < longitud:
            #    serie_con_cambios[fin_cambio:] += magnitud
    
    # Añadir ruido
    ruido = nivel_ruido * np.random.normal(0, 1, longitud)
    serie_final = serie_con_cambios + ruido
    
    # Graficar
    if graficar:
        plt.figure(figsize=(14, 8))
        plt.plot(serie_final, label='Serie con ruido y cambios', alpha=0.8, 
                linewidth=1.5, color='blue', zorder=2)
        plt.plot(serie_base, label='Tendencia base (sin cambios)', alpha=0.8, 
                linewidth=2, color='green', linestyle='--', zorder=3)
        plt.plot(serie_con_cambios, label='Tendencia con cambios (sin ruido)', 
                alpha=0.7, linewidth=2, color='red', zorder=1)
        
        y_min, y_max = plt.ylim()
        for i, punto in enumerate(puntos_cambio):
            plt.axvline(x=punto, color='purple', linestyle='--', alpha=0.8, 
                       linewidth=1.5, zorder=1)
            plt.text(punto, y_max * 0.92, f'Cambio {i+1}', rotation=90, 
                    verticalalignment='top', fontweight='bold', fontsize=10,
                    bbox=dict(boxstyle="round,pad=0.3", facecolor="yellow", 
                             alpha=0.7), zorder=4)
        
        plt.title(f'Serie de Tiempo Sintética\n(Ruido: {nivel_ruido}, Cambios: {num_cambios}, Fuerza: {fuerza_cambio})', 
                 fontsize=14, fontweight='bold')
        plt.xlabel('Tiempo', fontsize=12)
        plt.ylabel('Valor', fontsize=12)
        plt.legend(loc='upper left', fontsize=10)
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()
    
    return serie_final, puntos_cambio





import numpy as np
import pandas as pd
import json
import os

def generar_series_unico_csv_json(
    configuraciones,
    ruta_csv="series.csv",
    num_series_por_tipo=50,
    longitud=500,
    tipo_cambio='pendiente',
    seed=None,
    
):
    if seed is not None:
        np.random.seed(seed)

    df_series = pd.DataFrame()
    change_points_dict = {}



    serie_id = 1
    for idx_tipo, config in enumerate(configuraciones, 1):
        for _ in range(num_series_por_tipo):
            # Número de cambios aleatorio entre 1 y 3
            num_cambios = np.random.randint(1, 4)
            
            # Fuerza de cambio y ruido aleatorio dentro del rango
            fuerza_cambio = np.random.uniform(*config["fuerza_cambio"])
            nivel_ruido = np.random.uniform(*config["nivel_ruido"])

            # Generar serie usando tu función original
            serie, puntos_cambio = generar_serie_sintetica(
                longitud=longitud,
                nivel_ruido=nivel_ruido,
                num_cambios=num_cambios,
                fuerza_cambio=fuerza_cambio,
                tipo_cambio=tipo_cambio,
                graficar=False
            )

            # Nombre de columna
            col_name = f"tipo{idx_tipo}_serie{serie_id}"
            df_series[col_name] = serie

            # Guardar puntos de cambio en diccionario usando el nombre de la columna
            change_points_dict[col_name] = [int(x) for x in puntos_cambio]
            
            '''
            change_points_dict[col_name] = {
                "tipo_serie": idx_tipo,
                "puntos_cambio": [int(x) for x in puntos_cambio]  # Convertir a int nativo
            }
            '''
            serie_id += 1

    # Guardar CSV
    df_series.to_csv(ruta_csv, index=False)


    return df_series, change_points_dict

In [ ]:
# Definición de los 4 tipos de series de acuerdo a la fuerza de cambio y el nivel de ruido. Sin embargo esto se puede hacer para ruido con tipo de 
# cambio escalon, es decir cambios en la media, o ruido con cambios en la pendiente. De esta manera hablamos de 8 grupos. 
configuraciones = [
        {"fuerza_cambio": (3, 6), "nivel_ruido": (3, 6)},
        {"fuerza_cambio": (3, 6), "nivel_ruido": (0, 0.3)},
        {"fuerza_cambio": (0.3, 1), "nivel_ruido": (3, 6)},
        {"fuerza_cambio": (0.3, 1), "nivel_ruido": (0, 0.3)},
    ]

ruta_changepoints = "C:/MATERIAL TRABAJO/CIENCIA DE DATOS PROYECTOS/CRIME PREDICTION/changePoint/sinteticos/change_points_escalon.json"

df_series, change_points_dict= generar_series_unico_csv_json(configuraciones, num_series_por_tipo=50, longitud=200, tipo_cambio="escalon", ruta_csv="C:/MATERIAL TRABAJO/CIENCIA DE DATOS PROYECTOS/CRIME PREDICTION/changePoint/sinteticos/todas_series_escalon.csv")


# Convertir todos los elementos a int o float
change_points_dict_python = {k: [int(x) for x in v] for k, v in change_points_dict.items()}

with open(ruta_changepoints, "w") as f:
    json.dump(change_points_dict_python, f, indent=4)